In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Loading the road accidents data 
road_acc = pd.read_csv('/Users/elorm/Documents/Repos/Datasets/road-accidents.csv', sep = '|', comment = '#')

#Loading the miles driven data
miles = pd.read_csv('/Users/elorm/Documents/Repos/Datasets/miles-driven.csv')

#Inspecting the first few rows of the datasets
road_acc.head(3)

,state,drvr_fatl_col_bmiles,perc_fatl_speed,perc_fatl_alcohol,perc_fatl_1st_time
0,Alabama,18.8,39,30,80
1,Alaska,18.1,41,25,94
2,Arizona,18.6,35,28,96


`drvr_fatl_col_bmiles` = Number of drivers involved in fatal collisions per billion miles (2011)',
`perc_fatl_speed` = Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding (2009)',
`perc_fatl_alcohol` = Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired (2011)',
`perc_fatl_1st_time` = Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents (2011)